# import librairies

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# download mails

In [23]:
import os
import sys
sys.path.insert(0, 'put_absolute_path_of_mailautolabel_module')

from mailautolabel.imap.imap import IMAPMain

hostname=''
username=''
password=''

imap = IMAPMain(hostname=hostname, username=username, password=password)
messages = imap.get_messages(
    header_keys=['from', 'subject', 'date', 'from', 'to']
)

# save mails into a csv

In [24]:
import pandas, os
path_to_save = os.path.abspath('../data/{}.csv'.format(username))
pandas.DataFrame(messages).to_csv(path_to_save)

# read mails from csv

In [25]:
df = pd.read_csv(os.path.abspath('../data/{}.csv'.format(username)), sep=",", engine="python", header=0)

# show first matrix lines

In [ ]:
np.transpose(df.head())

# split train_test

In [ ]:
X = df.drop(['folder'], axis=1)
y = pd.DataFrame(df['folder'])

test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

print('X : {}\nY : {}'.format(X.shape, y.shape))
print('X_train : {}\nX_test : {}\ny_train : {}\ny_test : {}'.format(
    X_train.shape, X_test.shape, y_train.shape, y_test.shape))

In [ ]:
X_train['body']

# TF-IDF vectorizer

In [ ]:
vectorizer = TfidfVectorizer(analyzer="word")
X_train_vec = vectorizer.fit_transform(X_train['body'])
X_train_vec = pd.DataFrame(X_train_vec.todense(), columns=vectorizer.get_feature_names())

X_test_vec = vectorizer.transform(X_test['body'])
X_test_vec = pd.DataFrame(X_test_vec.todense(), columns=vectorizer.get_feature_names())
X_test_vec.shape

print('X_train_vec {}\nX_test_vec {}'.format(X_train_vec.shape, X_test_vec.shape))

# Logistic regression

In [ ]:
classifier = LogisticRegression(class_weight='balanced')
#y_train = np.ravel(y_train)
classifier.fit(X_train_vec, y_train)

In [ ]:
predicts = classifier.predict(X_test_vec)
f1_score(y_test, predicts, average='macro')

In [ ]:
X_test.shape
y_test.shape

y_train.shape

# Random forest

In [ ]:
### RANDOMFORESTCLASSIFIER
rf_clf = RandomForestClassifier(n_estimators=200)
rf_clf.fit(X_train_vec, y_train)

print("Taux de bonne classification : %f" %rf_clf.score(X_test_vec, y_test))
# Calcul du F-score
y_predits = rf_clf.predict(X_test_vec)
print("F-score global : %f " %f1_score(y_test, y_predits, average='micro'))
print("F-score global (déséquilibré) : %f " %f1_score(y_test, y_predits, average='macro'))
F_scores = f1_score(y_test, y_predits, average=None)
F_scores

In [ ]:
np.column_stack((X_test['Message-ID'], predicts))